In [2]:
import plotly.graph_objects as go
import copy_ as cp
dt = 'date_timestamp'
import psycopg2;import pandas as pd;import process as dp;import functions_straddle as fn;from pandas.errors import SettingWithCopyWarning;import GeeksCalculator as gk;import importlib;import warnings;importlib.reload(dp);importlib.reload(cp);importlib.reload(fn);importlib.reload(gk);warnings.simplefilter(action='ignore', category=FutureWarning);warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
conn = psycopg2.connect(dbname="qdap_test",user="amt",password="your_password",host="192.168.2.23",port="5432")
cursor = conn.cursor()

ImportError: cannot import name 'onIronPython' from 'adodbapi.apibase' (C:\Users\Tanmay\AppData\Roaming\Python\Python312\site-packages\adodbapi\apibase.py)

In [1]:
importlib.reload(dp); importlib.reload(fn)
data = []
grid = [(9,15),(9,30),(9,45),(10,00),(10,15)]
for eh,em in grid:
    print(eh,em)
    ranges = pd.date_range(start='2020-01-01', end='2024-06-30')
    # entry time
    tradeid = 0
    DGrid = [(0.1,0.55),(0.2,0.55),(0.3,0.55),(0.4,0.55)]
    for dl,du in DGrid:
        all_trades, all_execs , trade_id = {} , {} ,0
        all_running_pnl, all_call_movement, all_put_movement, all_call_iv, all_put_iv = [],[],[],[],[]
        list_symbol = ['NIFTY']
        for symbol in list_symbol:
            all_trades[symbol],all_execs[symbol] = [] , []
            for d_t in ranges:
                try:
                    futures = dp.FetchFuturesByDate(cursor,symbol,d_t)
                    if not futures.empty:
                        options = dp.FetchOptionsByDateCsv(symbol,d_t.date())
                        # print(symbol,str(d_t.date()))
                        gen_call, gen_put = options[options['opt_type'] == 'CE'], options[options['opt_type'] == 'PE']
                        call, put = dp.fetch_call_put_(options)
                        call.ffill(inplace=True) ; put.ffill(inplace=True)
                        tradeid,execs, trades, running_pnl, difference, delta_put, delta_call, iv, ivs = fn.generate_trades_syn(trade_id,eh,em,futures, gen_call, gen_put, io=0, buying='open', selling='close', DeltaLB=0.2,delta_range=0.1, DeltaUB = 0.5,initial_capital=100000)
                        all_running_pnl.extend(running_pnl);   all_trades[symbol].extend(trades);  all_execs[symbol].extend(execs)
                        if len(trades) == 0:
                            print(f"No trades took place on {d_t.date()}")
                except Exception as e:
                    print(f"Error on date {d_t} for symbol {symbol}: {str(e)}")
        
        print('EntryTime :',eh,em,'Lower_delta : ',dl,'Upper_delta : ',du)
        sharpie, stats = fn.sharpie_straddle(all_trades['NIFTY'])
        print("sharpie :",sharpie)
        x = fn.calculate_pnl_metrics(stats)
        print('monthly_avg_loss',x['monthly_avg_loss'])
        print('monthly_avg_profit',x['monthly_avg_profit'])
        print('total pnl ',x['total_pnl'])
        print('monthly_avg_profit/monthly_avg_loss',x['monthly_avg_profit/monthly_avg_loss'])
        print('max_pnl',x['max_pnl'])
        print('min_pnl',x['min_pnl'])
        print('win_ratio',x['win_ratio'])
        data.append({'EntryTime':(eh,em), 'DeltaLB':dl,'DeltaUB':du,'sharpie':sharpie,'mal':x['monthly_avg_loss'],'map':x['monthly_avg_profit'],'total_pnl':x['total_pnl'],'MPL_Ratio':x['monthly_avg_profit/monthly_avg_loss'] , 'max_pnl':x['max_pnl'],'min_pnl':x['min_pnl'],'win_ratio':x['win_ratio']})
    

NameError: name 'importlib' is not defined

In [71]:
dat = pd.DataFrame(data)
dat.to_csv('a.csv')

In [40]:
pri_ = {'id':1,'time': 1,'opt_type':1,'log':1,'delta':1,'strike': 1, 'price': 1,'pnl': 0, 'trade_type': 1,'transaction_cost': 0, 'slippage': 0,}
fn.print_execs_to_html(all_execs['NIFTY'],pri_,filename=f'all exec log.html')

columns_to_print = {'entry_time':1 , 'entry_price':1 ,'exit_price':1, 'exit_time' : 1 ,  'time':1 , 'movement': 0, 'type':1 , 'pnl':1,'strike':1,'entry_delta':1,'exit_delta':1,'sell_id':1,'buy_id':1}
fn.print_trades_to_html(all_trades['NIFTY'], columns_to_print , filename=f'all trade logs.html')

sharpie, stats = fn.sharpie_straddle(all_trades['NIFTY'])
print(sharpie)


C:\Users\Tanmay\Desktop\Files\all exec log.html

C:\Users\Tanmay\Desktop\Files\all trade logs.html

0.5108112107591711


In [47]:
print(stats)

            daily_pnl  daily_capital  daily_return
exit_date                                         
2020-01-01       2.65            312      0.008494


In [41]:
sharpie1,stats = fn.sharpie_straddle_Combine(all_trades)

In [67]:
print(stats)

            daily_pnl  daily_capital  daily_return
exit_date                                         
2020-01-03     -0.804            187     -0.004299
2020-01-06      5.687            302      0.018831
2020-01-07    -20.792            658     -0.031599
2020-01-08     14.165            651      0.021759
2020-01-10     -8.111            494     -0.016419
...               ...            ...           ...
2024-06-21    -34.693           1803     -0.019242
2024-06-24     -6.761            644     -0.010498
2024-06-25    -16.283            538     -0.030266
2024-06-26      0.720            761      0.000946
2024-06-28      9.609           1014      0.009476

[751 rows x 3 columns]


In [43]:
importlib.reload(dp); importlib.reload(fn)
x = fn.calculate_pnl_metrics(stats)
print('monthly_avg_loss',x['monthly_avg_loss'])
print('monthly_avg_profit',x['monthly_avg_profit'])
print('total pnl ',x['total_pnl'])
print('monthly_avg_profit/monthly_avg_loss',x['monthly_avg_profit/monthly_avg_loss'])
print('max_pnl',x['max_pnl'])
print('min_pnl',x['min_pnl'])
print('win_ratio',x['win_ratio'])

monthly_avg_loss -7.713642857142858
monthly_avg_profit 10.813318181818182
total pnl  16.672999999999995
monthly_avg_profit/monthly_avg_loss -1.4018432512473682
max_pnl 22.811
min_pnl -20.407999999999998
win_ratio 1.3


In [9]:
class Trade:
    def __init__(self,id,dt,price,strike,tradetype,delta,log):
        self.DT = dt
        self.ID = id
        self.Price = price
        self.Strike = strike
        self.Delta = delta
        self.TradeType = tradetype
        self.Log = log

In [13]:
ar = []
ar.append(Trade(1,2,3,4,5,6,7))
print(ar[0].Log)

7
